## Concept Questions
**What is a decorator in Python, and where is it used?**  
    a function takes another function as input, to enhance its behavior without changing its code, it's a higher-order function (HOF) basically.  
    Use cases: logging, authentication, caching, timing function execution, etc.

In [ ]:
def decorator(func):
    def wrapper():
        print("Before function")
        func()
        print("After function")
    return wrapper

@decorator
def say_hello():
    print("Hello")

say_hello()

**What's the difference between a generator and a regular function that returns a list?**
- Regular function: builds the entire list in memory and returns it.
- Generator: yields items one by one without storing the entire sequence in memory.

In [ ]:
def regular():
    return [i for i in range(1000000)]

def gen():
    for i in range(1000000):
        yield i

**When would you choose generators over lists, and what are the memory implications?**
- Use generators when handling large datasets or streams of data, like reading a file line by line.
- lists store all elements, and generators produce elements one-by-one, saving memory.

**Explain the difference between threading, multiprocessing, and asyncio in Python**
- Threading: Runs multiple threads in one process; good for I/O-bound tasks; share the same memory space, so limited by the GIL.
- Multiprocessing: Runs multiple processes; achieves true parallelism; best for CPU-bound tasks; each process runs in a seperate memory space, so bypassing the GIL.
- Asyncio: Uses a single thread with an event loop; great for high-concurrency I/O with async/await syntax.

**What is the Global Interpreter Lock (GIL)? How does it affect threading and multiprocessing?**
- Python’s GIL allows only one thread to execute Python bytecode at a time.
- Threading: limited for CPU-bound tasks because of GIL.
- Multiprocessing: bypasses GIL because each process has its own interpreter.

**When to use threading, asyncio, multiprocess?**
- Threading: multiple IO-bound tasks, e.g., web requests.
- Asyncio: massive number of concurrent IO-bound tasks, e.g., web scraping.
- Multiprocessing: CPU-heavy tasks, e.g., image processing, numerical computations.

**What are CPU-bound vs IO-bound tasks?**
- CPU-bound: task that uses the CPU intensively. Example: calculations, data processing.
- IO-bound: task that waits on external resources. Example: file read/write, network requests.

**What's the difference between yield and return in a function**
- return: exits function and gives a value (single output).
- yield: pauses function, returns a value, and resumes from where it left off. Used in generators.

**What's the difference between using open() with explicit close() vs using the with statement**
- with is safer because it automatically closes, even if exceptions occur.

In [ ]:
# Explicit close
f = open("file.txt")
data = f.read()
f.close()

# With statement
with open("file.txt") as f:
    data = f.read()
# Automatically closes

**How to handle exceptions? Why is exception handling important?**  
- Importance: prevents crashes, allows graceful handling, logging, retries.  
- Use try-except blocks:

In [ ]:
try:
    result = 10 / 0
except ZeroDivisionError:
    result = 0

## Coding Questions
### Coding Problem 1: Docorator

**Problem:**  
Decorator to cache any function return and log hits/misses

**Description:**  
Create a decorator that:
* Caches results for any function with any arguments. (Cache means returning the result directly without calling the function)
* Logs when the function is called
* Logs cache hits
* Logs cache misses

-> whether the function with same arguments has been called before

In [2]:
def cache_with_log(func):
    """
    Cache decorator that logs all activity.
    Should work with any function signature.
    Each function has its own cache dictionary.
    """
    cache = {}
    def wrapper(*args, **kwargs):
        key = args + tuple(sorted(kwargs.items()))
        print('key: ', key)
        if key in cache:
            print(f'[Cache HIT] {func.__name__} called with args={args}, kwargs={kwargs}')
            return cache[key]
        else:
            print(f'[Cache MISS] {func.__name__} called with args={args}, kwargs={kwargs}')
            result = func(*args, *kwargs)
            cache[key] = result
            return result
    
    return wrapper

# Test cases
@cache_with_log
def add(a, b):
    """Simple function with positional args"""
    return a + b

@cache_with_log
def greet(name, greeting="Hello"):
    """Function with keyword args"""
    return f"{greeting}, {name}!"

@cache_with_log
def calculate(x, y, operation="add"):
    """Function with mixed args"""
    if operation == "add":
        return x + y
    elif operation == "multiply":
        return x * y


# Run tests
print("=== Test 1: Simple function ===")
print(add(2, 3))      # Should log: Cache MISS for args={args}, kwargs={kwargs}
print(add(2, 3))      # Should log: Cache HIT for args={args}, kwargs={kwargs}
print(add(4, 5))      # Should log: Cache MISS for args={args}, kwargs={kwargs}

print("\n=== Test 2: Function with kwargs ===")
print(greet("Alice"))                    # Should log: Cache MISS for args={args}, kwargs={kwargs}
print(greet("Alice"))                    # Should log: Cache HIT for args={args}, kwargs={kwargs}
print(greet("Bob", greeting="Hi"))       # Should log: Cache MISS for args={args}, kwargs={kwargs}
print(greet("Bob", greeting="Hi"))       # Should log: Cache HIT for args={args}, kwargs={kwargs}

print("\n=== Test 3: Mixed args ===")
print(calculate(3, 4))                   # Should log: Cache MISS for args={args}, kwargs={kwargs}
print(calculate(3, 4, operation="add"))  # Should log: Cache HIT for args={args}, kwargs={kwargs}
print(calculate(3, 4, operation="multiply"))  # Should log: Cache MISS for args={args}, kwargs={kwargs}


=== Test 1: Simple function ===
key:  (2, 3)
[Cache MISS] add called with args=(2, 3), kwargs={}
5
key:  (2, 3)
[Cache HIT] add called with args=(2, 3), kwargs={}
5
key:  (4, 5)
[Cache MISS] add called with args=(4, 5), kwargs={}
9

=== Test 2: Function with kwargs ===
key:  ('Alice',)
[Cache MISS] greet called with args=('Alice',), kwargs={}
Hello, Alice!
key:  ('Alice',)
[Cache HIT] greet called with args=('Alice',), kwargs={}
Hello, Alice!
key:  ('Bob', ('greeting', 'Hi'))
[Cache MISS] greet called with args=('Bob',), kwargs={'greeting': 'Hi'}
greeting, Bob!
key:  ('Bob', ('greeting', 'Hi'))
[Cache HIT] greet called with args=('Bob',), kwargs={'greeting': 'Hi'}
greeting, Bob!

=== Test 3: Mixed args ===
key:  (3, 4)
[Cache MISS] calculate called with args=(3, 4), kwargs={}
7
key:  (3, 4, ('operation', 'add'))
[Cache MISS] calculate called with args=(3, 4), kwargs={'operation': 'add'}
None
key:  (3, 4, ('operation', 'multiply'))
[Cache MISS] calculate called with args=(3, 4), kwargs=

### Coding Problem 2: Batch Generator

**Problem:**  
Create a generator that takes an iterable and yields items in batches of a specified size.


In [4]:
def batch(iterable, batch_size):
    """
    Generator that yields items in batches.
    
    Args:
        iterable: Any iterable (list, generator, etc.)
        batch_size: Number of items per batch
    
    Yields:
        Lists of items with length = batch_size (last batch may be smaller)
    
    Example:
        >>> list(batch([1, 2, 3, 4, 5, 6, 7], 3))
        [[1, 2, 3], [4, 5, 6], [7]]
        
        >>> list(batch(range(10), 4))
        [[0, 1, 2, 3], [4, 5, 6, 7], [8, 9]]
    """

    batch_list = []
    for item in iterable:
        batch_list.append(item)
        if len(batch_list) == batch_size:
            yield batch_list
            batch_list = []
    if batch_list:
        # the remaining items
        yield batch_list


# Test cases
print(list(batch([1, 2, 3, 4, 5, 6, 7], 3)))
# [[1, 2, 3], [4, 5, 6], [7]]

print(list(batch(range(10), 4)))
# [[0, 1, 2, 3], [4, 5, 6, 7], [8, 9]]

print(list(batch("ABCDEFGH", 2)))
# [['A', 'B'], ['C', 'D'], ['E', 'F'], ['G', 'H']]

[[1, 2, 3], [4, 5, 6], [7]]
[[0, 1, 2, 3], [4, 5, 6, 7], [8, 9]]
[['A', 'B'], ['C', 'D'], ['E', 'F'], ['G', 'H']]


### Coding Problem 3: Async Retry with Exponential Backoff

**Problem**
Create a decorator that retries async functions with exponential backoff.

In [8]:
!python q3_async.py

Test 1: Flaky API (30% success rate)
  Attempting API call...
Failed!
[Attempt 1] Failed: API temporarily unavailable. Retrying in 0.50s...
  Attempting API call...
Failed!
[Attempt 2] Failed: API temporarily unavailable. Retrying in 1.00s...
  Attempting API call...
Failed!
[Attempt 3] Failed: API temporarily unavailable. Retrying in 2.00s...
  Attempting API call...
Success!
Final result: API response data

Test 2: Very flaky API (10% success rate)
  Attempting API call...
Failed!
[Attempt 1] Failed: API temporarily unavailable. Retrying in 0.50s...
  Attempting API call...
Failed!
[Attempt 2] Failed: API temporarily unavailable. Retrying in 1.00s...
  Attempting API call...
Failed!
[Attempt 3] Failed: API temporarily unavailable. Retrying in 2.00s...
  Attempting API call...
Failed!
[Attempt 4] Failed: API temporarily unavailable. Retrying in 4.00s...
  Attempting API call...
Success!
Final result: API response data

